In [1]:
# 필수 패키지 설치
# pip install python-pptx openai python-dotenv

import os
from typing import List, Dict, Optional
from pptx import Presentation
from pptx.util import Pt, Inches
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
import re
from pathlib import Path

# 환경 변수 로드
load_dotenv()


True

In [2]:
class AzureOpenAITranslator:
    """Azure OpenAI를 사용한 번역 엔진"""
    
    def __init__(
        self,
        api_key: str = None,
        api_version: str = "2025-01-01-preview",
        azure_endpoint: str = None,
        deployment_name: str = "gpt-4.1",
        temperature: float = 0.1,
        max_tokens: int = 4000
    ):
        """
        Azure OpenAI 번역기 초기화
        
        Args:
            api_key: Azure OpenAI API 키
            api_version: Azure OpenAI API 버전
            azure_endpoint: Azure OpenAI 엔드포인트 URL
            deployment_name: 배포된 모델 이름 (예: gpt-4, gpt-35-turbo)
            temperature: 생성 온도 (0.0 ~ 1.0)
            max_tokens: 최대 토큰 수
        """
        self.api_key = api_key or os.getenv("AZURE_OPENAI_API_KEY")
        self.azure_endpoint = azure_endpoint or os.getenv("AZURE_OPENAI_ENDPOINT")
        self.deployment_name = deployment_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        
        # Azure OpenAI 클라이언트 초기화
        self.client = AzureOpenAI(
            api_key=self.api_key,
            api_version=api_version,
            azure_endpoint=self.azure_endpoint
        )
    
    def translate_batch(
        self,
        texts: List[str],
        target_language: str = "ko",
        enable_polishing: bool = True
    ) -> List[str]:
        """
        텍스트 배치 번역
        
        Args:
            texts: 번역할 텍스트 리스트
            target_language: 목표 언어 코드
            enable_polishing: 자연스러운 표현 개선 여부
        
        Returns:
            번역된 텍스트 리스트
        """
        if not texts:
            return []
        
        # 번역 프롬프트 생성
        prompt = self._create_translation_prompt(texts, target_language, enable_polishing)
        
        try:
            # Azure OpenAI API 호출
            response = self.client.chat.completions.create(
                model=self.deployment_name,
                messages=[
                    {
                        "role": "system",
                        "content": "You are a professional translator specializing in PowerPoint presentations."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=self.temperature,
                max_tokens=self.max_tokens
            )
            
            # 응답 파싱
            translated_text = response.choices[0].message.content
            return self._parse_translation_response(translated_text, len(texts))
            
        except Exception as e:
            print(f"Translation error: {str(e)}")
            return texts  # 오류 발생 시 원본 반환
    
    def _create_translation_prompt(
        self,
        texts: List[str],
        target_language: str,
        enable_polishing: bool
    ) -> str:
        """번역 프롬프트 생성"""
        
        language_map = {
            "ko": "Korean",
            "en": "English",
            "ja": "Japanese",
            "zh": "Chinese (Simplified)",
            "zh-tw": "Chinese (Traditional)",
            "es": "Spanish",
            "fr": "French",
            "de": "German",
            "pt": "Portuguese"
        }
        
        target_lang = language_map.get(target_language, target_language)
        
        polishing_instruction = ""
        if enable_polishing:
            polishing_instruction = """
- Make the translation natural and fluent in the target language
- Use appropriate business/technical terminology
- Maintain the tone and style of the original text
"""
        
        # 텍스트를 JSON 형식으로 준비
        texts_json = json.dumps(texts, ensure_ascii=False, indent=2)
        
        prompt = f"""Translate the following texts to {target_lang}.

Instructions:
- Translate each text accurately while preserving its meaning
- Keep the order of texts unchanged
- Preserve any formatting markers or special characters
- If text contains technical terms, keep them accurate
{polishing_instruction}
- Return ONLY a JSON array of translated texts, nothing else

Input texts:
{texts_json}

Output format (JSON array only):
["translated text 1", "translated text 2", ...]
"""
        
        return prompt
    
    def _parse_translation_response(self, response: str, expected_count: int) -> List[str]:
        """번역 응답 파싱"""
        try:
            # JSON 배열 추출
            json_match = re.search(r'\[.*\]', response, re.DOTALL)
            if json_match:
                translations = json.loads(json_match.group())
                if len(translations) == expected_count:
                    return translations
            
            # JSON 파싱 실패 시 줄바꿈으로 분리 시도
            lines = [line.strip() for line in response.split('\n') if line.strip()]
            if len(lines) == expected_count:
                return lines
            
        except json.JSONDecodeError:
            pass
        
        # 파싱 실패 시 응답을 그대로 반환
        return [response]


In [3]:
class PowerPointTranslator:
    """PowerPoint 번역 핸들러"""
    
    def __init__(self, translator: AzureOpenAITranslator):
        """
        PowerPoint 번역기 초기화
        
        Args:
            translator: AzureOpenAITranslator 인스턴스
        """
        self.translator = translator
        self.font_map = {
            "ko": "맑은 고딕",
            "ja": "Yu Gothic UI",
            "en": "Arial",
            "zh": "Microsoft YaHei",
            "zh-tw": "Microsoft JhengHei"
        }
    
    def extract_texts_from_slide(self, slide) -> List[Dict]:
        """슬라이드에서 텍스트 추출"""
        texts_data = []
        
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                if hasattr(shape, "text_frame"):
                    for paragraph in shape.text_frame.paragraphs:
                        for run in paragraph.runs:
                            if run.text.strip():
                                texts_data.append({
                                    "shape": shape,
                                    "paragraph": paragraph,
                                    "run": run,
                                    "text": run.text,
                                    "font_size": run.font.size,
                                    "font_name": run.font.name,
                                    "bold": run.font.bold,
                                    "italic": run.font.italic,
                                    "color": run.font.color.rgb if run.font.color.type == 1 else None
                                })
        
        return texts_data
    
    def translate_presentation(
        self,
        input_path: str,
        output_path: str,
        target_language: str = "ko",
        slide_numbers: Optional[List[int]] = None,
        batch_size: int = 20,
        enable_polishing: bool = True
    ):
        """
        PowerPoint 프레젠테이션 번역
        
        Args:
            input_path: 입력 PPTX 파일 경로
            output_path: 출력 PPTX 파일 경로
            target_language: 목표 언어 코드
            slide_numbers: 번역할 슬라이드 번호 리스트 (None이면 전체)
            batch_size: 배치 번역 크기
            enable_polishing: 자연스러운 표현 개선 여부
        """
        # 프레젠테이션 로드
        prs = Presentation(input_path)
        total_slides = len(prs.slides)
        
        print(f"총 슬라이드 수: {total_slides}")
        
        # 번역할 슬라이드 결정
        if slide_numbers:
            slides_to_translate = [i for i in slide_numbers if 1 <= i <= total_slides]
            print(f"번역할 슬라이드: {slides_to_translate}")
        else:
            slides_to_translate = list(range(1, total_slides + 1))
            print("전체 슬라이드 번역")
        
        # 각 슬라이드 번역
        for slide_idx, slide in enumerate(prs.slides, 1):
            if slide_idx not in slides_to_translate:
                print(f"슬라이드 {slide_idx} 건너뛰기")
                continue
            
            print(f"\n슬라이드 {slide_idx}/{total_slides} 번역 중...")
            
            # 텍스트 추출
            texts_data = self.extract_texts_from_slide(slide)
            
            if not texts_data:
                print(f"  텍스트 없음")
                continue
            
            # 배치 번역
            texts_to_translate = [item["text"] for item in texts_data]
            
            print(f"  번역할 텍스트 수: {len(texts_to_translate)}")
            
            for i in range(0, len(texts_to_translate), batch_size):
                batch = texts_to_translate[i:i + batch_size]
                translated_batch = self.translator.translate_batch(
                    batch,
                    target_language=target_language,
                    enable_polishing=enable_polishing
                )
                
                # 번역된 텍스트 적용
                for j, translated_text in enumerate(translated_batch):
                    idx = i + j
                    if idx < len(texts_data):
                        text_item = texts_data[idx]
                        text_item["run"].text = translated_text
                        
                        # 목표 언어에 맞는 폰트 적용
                        target_font = self.font_map.get(target_language, "Arial")
                        text_item["run"].font.name = target_font
                
                print(f"  배치 {i//batch_size + 1} 완료")
        
        # 저장
        prs.save(output_path)
        print(f"\n번역 완료! 저장 경로: {output_path}")
    
    def get_slide_info(self, input_path: str):
        """슬라이드 정보 조회"""
        prs = Presentation(input_path)
        
        print(f"총 슬라이드 수: {len(prs.slides)}\n")
        
        for slide_idx, slide in enumerate(prs.slides, 1):
            print(f"슬라이드 {slide_idx}:")
            
            text_count = 0
            for shape in slide.shapes:
                if hasattr(shape, "text") and shape.text.strip():
                    text_count += 1
                    print(f"  - {shape.text[:50]}{'...' if len(shape.text) > 50 else ''}")
            
            if text_count == 0:
                print("  (텍스트 없음)")
            
            print()


In [4]:
# ===========================
# 환경 설정
# ===========================

# 방법 1: 코드에서 직접 설정
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_NAME = "gpt-4.1"  # 또는 "gpt-35-turbo", "gpt-4-turbo" 등

# ===========================
# 번역기 초기화
# ===========================

translator = AzureOpenAITranslator(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    deployment_name=DEPLOYMENT_NAME,
    temperature=0.1,
    max_tokens=4000
)

ppt_translator = PowerPointTranslator(translator)

# ===========================
# 예제 1: 전체 슬라이드 번역
# ===========================

input_file = "ai_idea.pptx"
output_file = "ai_idea_ko.pptx"

ppt_translator.translate_presentation(
    input_path=input_file,
    output_path=output_file,
    target_language="ko",
    enable_polishing=True
)

# ===========================
# 예제 2: 특정 슬라이드만 번역
# ===========================

# ppt_translator.translate_presentation(
#     input_path=input_file,
#     output_path="presentation_selected_ko.pptx",
#     target_language="ko",
#     slide_numbers=[1, 3, 5, 7],  # 1, 3, 5, 7번 슬라이드만 번역
#     enable_polishing=True
# )

# # ===========================
# # 예제 3: 슬라이드 정보 확인
# # ===========================

# ppt_translator.get_slide_info(input_file)

# # ===========================
# # 예제 4: 다른 언어로 번역
# # ===========================

# # 영어로 번역
# ppt_translator.translate_presentation(
#     input_path=input_file,
#     output_path="presentation_en.pptx",
#     target_language="en",
#     enable_polishing=True
# )

# # 일본어로 번역
# ppt_translator.translate_presentation(
#     input_path=input_file,
#     output_path="presentation_ja.pptx",
#     target_language="ja",
#     enable_polishing=True
# )


총 슬라이드 수: 32
전체 슬라이드 번역

슬라이드 1/32 번역 중...
  번역할 텍스트 수: 1
  배치 1 완료

슬라이드 2/32 번역 중...
  번역할 텍스트 수: 4
  배치 1 완료

슬라이드 3/32 번역 중...
  번역할 텍스트 수: 15
  배치 1 완료

슬라이드 4/32 번역 중...
  번역할 텍스트 수: 3
  배치 1 완료

슬라이드 5/32 번역 중...
  번역할 텍스트 수: 8
  배치 1 완료

슬라이드 6/32 번역 중...
  번역할 텍스트 수: 5
  배치 1 완료

슬라이드 7/32 번역 중...
  번역할 텍스트 수: 6
  배치 1 완료

슬라이드 8/32 번역 중...
  번역할 텍스트 수: 2
  배치 1 완료

슬라이드 9/32 번역 중...
  번역할 텍스트 수: 4
  배치 1 완료

슬라이드 10/32 번역 중...
  번역할 텍스트 수: 7
  배치 1 완료

슬라이드 11/32 번역 중...
  번역할 텍스트 수: 13
  배치 1 완료

슬라이드 12/32 번역 중...
  번역할 텍스트 수: 14
  배치 1 완료

슬라이드 13/32 번역 중...
  번역할 텍스트 수: 5
  배치 1 완료

슬라이드 14/32 번역 중...
  번역할 텍스트 수: 3
  배치 1 완료

슬라이드 15/32 번역 중...
  번역할 텍스트 수: 22
  배치 1 완료
  배치 2 완료

슬라이드 16/32 번역 중...
  번역할 텍스트 수: 29
  배치 1 완료
  배치 2 완료

슬라이드 17/32 번역 중...
  번역할 텍스트 수: 7
  배치 1 완료

슬라이드 18/32 번역 중...
  번역할 텍스트 수: 2
  배치 1 완료

슬라이드 19/32 번역 중...
  번역할 텍스트 수: 5
  배치 1 완료

슬라이드 20/32 번역 중...
  번역할 텍스트 수: 6
  배치 1 완료

슬라이드 21/32 번역 중...
  번역할 텍스트 수: 5
  배치 1 완료

슬라이드 22/32 번역 